In [2]:
import httplib2
import os
import re

from apiclient import discovery
import oauth2client
from oauth2client import client
from oauth2client import tools

import datetime

# try:
#     import argparse
#     flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args()
# except ImportError:
flags = None

SCOPES = 'https://www.googleapis.com/auth/calendar'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'NuSTAR Calendar'

In [43]:
def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'calendar-quickstart.json')

    store = oauth2client.file.Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatability with Python 2.6
            credentials = tools.run(flow, store)
        print('Storing credentials to ' + credential_path)
    return credentials

In [44]:
"""Shows basic usage of the Google Calendar API.

Creates a Google Calendar API service object and outputs a list of the next
10 events on the user's calendar.
"""
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
service = discovery.build('calendar', 'v3', http=http)

now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time

print('Getting the upcoming 10 events')
eventsResult = service.events().list(
    calendarId='primary', timeZone='GMT',timeMin=now, maxResults=10, singleEvents=True,
    orderBy='startTime').execute()
events = eventsResult.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])
    

Getting the upcoming 10 events
2016-11-02T23:44:42Z 80201035001 TCP_J18102829m2729590
2016-11-02T23:50:04Z 80201035002 TCP_J18102829m2729590
2016-11-04T03:58:32Z 60201052001 PMN_J0948p0022
2016-11-04T04:05:15Z 60201052002 PMN_J0948p0022
2016-11-08T17:05:05Z 60201010001 PKS_0201p113
2016-11-08T17:15:02Z 60201010002 PKS_0201p113


In [45]:
dtm = start
dtm_spl = dtm.split('T')
print(dtm_spl)

date = dtm_spl[0].split('-')
time_fields = dtm_spl[1].split('-')
time = time_fields[0].split(':')
start_time = datetime.datetime.strptime(date[0]+' ' +date[1]+' '+date[2]+' '+time[0]+' '+time[1]+' '+time[2][:2], '%Y %m %d %H %M %S')

#start_time = datetime.datetime.strptime(dtm[0]+' ' +dtm[1]+' '+dtm[2]+' '+dtm[3]+' '+dtm[4], '%Y %j %H %M %S')
#print(dtm)


['2016-11-08', '17:15:02Z']


In [46]:
def add_event(start, stop, name):
    eventadd = service.events().quickAdd(
        calendarId='primary',
        text=name
        ).execute()
    eventadd['start']['dateTime']=start
    eventadd['end']['dateTime'] = stop
    updateEvent = service.events().update(
        calendarId='primary',eventId=eventadd['id'],
        body=eventadd
    ).execute()
    return

In [58]:
def cleanup_calendar(limitdays):

    now = datetime.datetime.utcnow()
    limit = (now + datetime.timedelta(-limitdays))
    limstr = limit.isoformat()+'Z'
    
    
    print('Removing the previous '+str(limitdays)+' days')
    eventsResult = service.events().list(
        calendarId='primary', timeMin=limstr, singleEvents=True,
        orderBy='startTime', timeZone='GMT').execute()
    events = eventsResult.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        
        dtm = start
        dtm_spl = dtm.split('T')
#        print(dtm_spl)

        date = dtm_spl[0].split('-')
        time_fields = dtm_spl[1].split('-')
        time = time_fields[0].split(':')
        start_time = datetime.datetime.strptime(date[0]+' ' +date[1]+' '+date[2]+' '+time[0]+' '+time[1]+' '+time[2][:2], '%Y %m %d %H %M %S')
        if (now - start_time).days > abs(limitdays):
            continue
        else:
            #print(start_time)
            #print((now - start_time).days)
        
        
        
        print('Removing: ',start, event['summary'])

        service.events().delete(
            calendarId='primary',
            eventId=event['id']).execute()

In [60]:
def populate_calendar(limit):
    
    now = datetime.datetime.utcnow()

    f = open('observing_schedule.txt', 'r')
    for line in f:
        if line.startswith(";"):
            continue
        fields = line.split()

        dtm = fields[0].split(':')
        start_time = datetime.datetime.strptime(dtm[0]+' ' +dtm[1]+' '+dtm[2]+' '+dtm[3]+' '+dtm[4], '%Y %j %H %M %S')

        dtm = fields[1].split(':')
        end_time = datetime.datetime.strptime(dtm[0]+' ' +dtm[1]+' '+dtm[2]+' '+dtm[3]+' '+dtm[4], '%Y %j %H %M %S')

        if (now - start_time).days > abs(limit):
            continue
        seqid = fields[2]
        seqname=fields[3]

        print('Adding: ', start_time.isoformat()+'Z', end_time.isoformat()+'Z', seqid+' '+seqname)
        add_event(start_time.isoformat()+'Z', end_time.isoformat()+'Z', seqid+' '+seqname)



In [62]:
cleanup_calendar(20)
populate_calendar(20)

Removing the previous 20 days
2016-10-13 06:11:00
20
Removing:  2016-10-13T06:11:00Z 60111040001 UDS_MOS040
2016-10-14 09:41:15
19
Removing:  2016-10-14T09:41:15Z 60111041001 UDS_MOS041
2016-10-15 13:05:00
18
Removing:  2016-10-15T13:05:00Z 60160148001 MCGp00m09m042
2016-10-15 13:15:09
18
Removing:  2016-10-15T13:15:09Z 60160148002 MCGp00m09m042
2016-10-16 02:03:39
17
Removing:  2016-10-16T02:03:39Z 90201039001 IC342_X1
2016-10-16 02:10:14
17
Removing:  2016-10-16T02:10:14Z 90201039002 IC342_X1
2016-10-17 03:14:06
16
Removing:  2016-10-17T03:14:06Z 60202015009 3C_382
2016-10-17 03:20:08
16
Removing:  2016-10-17T03:20:08Z 60202015010 3C_382
2016-10-17 15:13:34
16
Removing:  2016-10-17T15:13:34Z 40201014001 N157B
2016-10-17 15:20:22
16
Removing:  2016-10-17T15:20:22Z 40201014002 N157B
2016-10-19 14:52:56
14
Removing:  2016-10-19T14:52:56Z 80102101003 GRS_1739m278
2016-10-19 15:00:03
14
Removing:  2016-10-19T15:00:03Z 80102101004 GRS_1739m278
2016-10-20 05:59:38
13
Removing:  2016-10-20T0

In [25]:
now = datetime.datetime.utcnow()
limit = 15
f = open('observing_schedule.txt', 'r')
for line in f:
    if line.startswith(";"):
        continue
    fields = line.split()

    dtm = fields[0].split(':')
    start_time = datetime.datetime.strptime(dtm[0]+' ' +dtm[1]+' '+dtm[2]+' '+dtm[3]+' '+dtm[4], '%Y %j %H %M %S')

    dtm = fields[1].split(':')
    end_time = datetime.datetime.strptime(dtm[0]+' ' +dtm[1]+' '+dtm[2]+' '+dtm[3]+' '+dtm[4], '%Y %j %H %M %S')

    if (now - start_time).days > abs(limit):
        continue
    seqid = fields[2]
    seqname=fields[3]
    coords = line.find('[')
    qa_start = line.find('[', coords+1)
    qa_person=''
    if(qa_start != -1):
        qa_end =  line.find(']', qa_start)
        qa_person = line[qa_start+1:qa_end]

    print('Adding: ', start_time.isoformat()+'Z', end_time.isoformat()+'Z', seqid+' '+seqname+' ('+qa_person+')')
#    break
#    add_event(start_time.isoformat()+'Z', end_time.isoformat()+'Z', seqid+' '+seqname+' ('+qa_person+')')




Adding:  2016-11-15T13:20:51Z 2016-11-16T18:20:00Z 60111045001 UDS_MOS045 (Vikram)
Adding:  2016-11-14T08:20:40Z 2016-11-15T13:20:00Z 60111044001 UDS_MOS044 (Brian)
Adding:  2016-11-13T03:40:38Z 2016-11-14T08:20:00Z 60111043002 UDS_MOS043 (Hiro)
Adding:  2016-11-13T03:30:23Z 2016-11-13T03:40:00Z 60111043001 UDS_MOS043 ()
Adding:  2016-11-11T22:15:14Z 2016-11-13T02:55:00Z 30202002004 WR140 (Kristin)
Adding:  2016-11-11T22:06:25Z 2016-11-11T22:15:00Z 30202002003 WR140 ()
Adding:  2016-11-10T16:05:53Z 2016-11-11T21:30:00Z 60111042002 UDS_MOS042 (Javier)
Adding:  2016-11-10T15:55:15Z 2016-11-10T16:05:00Z 60111042001 UDS_MOS042 ()
Adding:  2016-11-09T13:15:05Z 2016-11-10T15:10:00Z 60202002002 NGC_3227 (Dan)
Adding:  2016-11-09T13:07:03Z 2016-11-09T13:15:00Z 60202002001 NGC_3227 ()
Adding:  2016-11-08T17:15:02Z 2016-11-09T12:15:00Z 60201010002 PKS_0201p113 (Vikram)
Adding:  2016-11-08T17:05:05Z 2016-11-08T17:15:00Z 60201010001 PKS_0201p113 ()
Adding:  2016-11-04T04:05:15Z 2016-11-08T16:15:00

In [16]:
print(line)


2016:320:13:20:51 2016:321:18:20:00 60111045001 UDS_MOS045 34.152296 -4.9218942 33.49333772 -5.34551792 150.72 [0.0,0.0] 1.0 18 55.0    Extragalactic Legacy Survey [Vikram]

164
Vikram
